---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [57]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind



# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [1]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [1]:
import pandas as pd
import numpy as np
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    with open('university_towns.txt') as file:
        utowns=file.readlines()
    data=[]
    town=0
    for line in utowns:
        university=line[:-1]
        if line[-7:]=='[edit]\n':
            town=line[:-7]
            continue
        elif '(' in line:
            university=line[:line.index('(')-1]
        else:
            university=line
        data.append((town,university))
    return pd.DataFrame(data,columns=['State','RegionName'])
    #return df
#get_list_of_university_towns()

In [3]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    #(skiprows starting from zero so use skiprow=np.arange(220))
    df=pd.read_excel('gdplev.xls',skiprows=220 ,usecols=[4,6],header=None,names=['Quarter','GDP'])
    recession=[]
    #i=0
    #while i<=len(df)-6:
    for i in range(len(df)-2):#as last two are missing for data of rest of year.
        if (df.iloc[i][1]>df.iloc[i+1][1]) and (df.iloc[i+1][1]>df.iloc[i+2][1]):
            #if (df.iloc[i+2][1]<df.iloc[i+3][1] and df.iloc[i+3][1]<df.iloc[i+4][1]):
            recession.append(df.iloc[i][0])
            
        
    return recession[1]
get_recession_start()

'2008q3'

In [4]:
def get_recession_end():
    start=get_recession_start()
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    df=pd.read_excel('gdplev.xls',skiprows=220 ,usecols=[4,6],header=None,names=['Quarter','GDP'])
    #recession_end=[] 
    #for i in range(len(df)-2):
    #    if (df.iloc[i][1] < df.iloc[i+1][1]) and (df.iloc[i+1][1] < df.iloc[i+2][1]):
    #        if df.iloc[i+2][1]<df.iloc[i+3][1] and df.iloc[i+3][1]< df.iloc[i+4][1])
    #        recession_end.append(df.iloc[i][0])
    
    df['Difference']=df['GDP'].diff()
    df=df[df['Quarter']>start]
    df= df[(df['Difference']>0) & (df['Difference'].shift(1)>0)] #use & not and        
    return df['Quarter'].iloc[0]       
get_recession_end()    

'2009q4'

In [26]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    #df=pd.read_excel('gdplev.xls',skiprows=220 ,usecols=[4,6],header=None,names=['Quarter','GDP'])
    #recession_start_index=get_recession_start()
    #recession_end_index=get_recession_end()
    #df.set_index('Quarter',drop=False)
    #return df.loc[recession_start_index:recession_end_index,'GDP'].idxmin()
    start=get_recession_start()
    df=pd.read_excel('gdplev.xls',skiprows=220 ,usecols=[4,6],header=None,names=['Quarter','GDP'])
    end=get_recession_end()    
    df=df[df['Quarter']> start]# was not able to use and here
    df=df[df['Quarter']<end]
    df=df.sort_values(by='GDP')
    
    return df['Quarter'].iloc[0]
get_recession_bottom()

#for i in range(0,len(df)-2,4):
    #    min=0
    #    quarter=df.iloc[i][0]
    #    for j in range(i,4,1):
    #        if min<df.iloc[j][1]:
    #            min=df.iloc[j][1]
    #            quarter=df.iloc[j][0]
    #    recession_bottom.append(quarter)

'2009q2'

In [2]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    city=pd.read_csv('City_Zhvi_AllHomes.csv')
    city['State']=city['State'].map(states)
    #city['State']=city['State'].apply(lambda x:states[x.State])
    city=city.set_index(['State','RegionName'])#inside a list
    city=city.loc[:,'2000-01':]
    city=city.loc[:,:'2016-12']
    new_cols=[str(x)+y for x in range(2000,2017) for y in ('q1','q2','q3','q4')]
    new_cols=new_cols[:-1]
    x=0
    for column in new_cols:
        city[column]=city.iloc[:,x:x+3].mean(axis=1)#use iloc not loc
        x=x+3
    city=city.loc[:,'2000q1':]#will include previous columns also so use indexing
    return city
#convert_housing_data_to_quarters()


In [71]:
from scipy import stats
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    start=get_recession_start()
    end=get_recession_end()
    housing_data=convert_housing_data_to_quarters()
    housing_data=housing_data.loc[:,start:end]
    housing_data.reset_index(inplace=True)
    housing_data['PriceRatio']=housing_data[start]-housing_data[end]/housing_data[start]
    university_towns=get_list_of_university_towns()
    unilist=university_towns['RegionName'].tolist()
    housing_data['isunitown']=housing_data['RegionName'].apply(lambda x:x in unilist)
    housing_data=housing_data.copy()
    university_data=housing_data[housing_data['isunitown']==True].dropna()# t-test can't be performed for na_values
    nonuniversity_data=housing_data[housing_data['isunitown']==False].dropna()
    p = ttest_ind(university_data['PriceRatio'], nonuniversity_data['PriceRatio'])[1]# here 1 is index position
    testing= p< 0.01
    if university_data['PriceRatio'].mean() < nonuniversity_data['PriceRatio'].mean():
        town='university town'
    else:
        town='non-university town'
    return (testing,p,town)
run_ttest()
    
    
   

(True, 3.9723617121333343e-06, 'university town')